In [1]:
import os
import logging
import pandas as pd
logger = logging.getLogger('LabJournal')
logging.basicConfig(level=logging.DEBUG)

In [2]:
import sys
# sys.path.append("../../..") # need this if labjournal is not installed

from labjournal.utils.fileFinder import FileFinder
from labjournal.core.databaseModel import *

In [3]:
SIMS = '/home/andrejb/Research/SIMS'

In [4]:
fileFinder = FileFinder(
    pattern = 'topol.tpr',
    path = SIMS,
    dir_ignore = ['data'])

INFO:LabJournal:FileFinder: pattern: topol.tpr
INFO:LabJournal:FileFinder: path: /home/andrejb/Research/SIMS
INFO:LabJournal:FileFinder: dir_ignore: ['data']


In [5]:
logger.info('create_database:create_database: FIND FILES')
ERRORS=False
WARNINGS=False

INFO:LabJournal:create_database:create_database: FIND FILES


In [13]:
SIM_IDS=[]
PATHS=[]
METAS=[]

for fname in fileFinder.find_files():
    path =  os.path.dirname(fname)
    sim_id = os.path.basename(path)
    meta = pd.Series.from_csv(os.path.join(path, "meta.csv"))
    SIM_IDS.append(sim_id)
    PATHS.append(path)
    METAS.append(meta)

/usr/local/lib/python2.7/dist-packages/pandas/core/series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [31]:
db = 'andrej_raw.db'
db = '/home/andrejb/Research/SIMS.db'
# remove old DB
try:
    os.remove(db)
    logger.info('create_database:create_database: removed old file: %s', db)
except:
    pass
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()
setup_database(engine)

In [32]:
for i, entry_id in enumerate(SIM_IDS):
    meta = METAS[i]
    try:
        description = meta['note']
    except KeyError:
        description = ""
    sim = Main(
        entry_id    = entry_id,
        path        = PATHS[i],
        sim_type    = "GROMACS",
        description = description
    )
    session.add(sim)
    session.flush()
    
    for k, v in meta.iteritems():
        if k != 'note':
            key = Keywords(
                main_id = sim.id,
                name = k,
                value = v
            )
            session.add(key)
session.commit()
session.close()